# Raspagem de Dados com Selenium

Raspar os dados do [ClimaTempo](https://www.climatempo.com.br/previsao-do-tempo/fim-de-semana/cidade/558/saopaulo-sp)

## Configuração do notebook

In [1]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from lib.database import Database
from selenium import webdriver
from datetime import datetime
import pandas as pd
import json
from sqlite3 import IntegrityError

DB_NAME = 'db.sqlite3'

# Funções auxiliares
def date_from_timestamp(timestamp):
    return timestamp.strftime('%Y-%m-%d')

def today():
    return datetime.today().strftime('%Y-%m-%d')

## Database Setup

- Inicializando os banco de dados

In [2]:
db = Database(DB_NAME)

- Carregando os locais no arquivo csv e armazenando-os no banco de dados
- Não serão inseridos novamente locais com o mesmo nome

In [3]:
with db as connection:
    with open('data/locais.csv') as file:
        db.load_locais(file)

## Web Scraping com Selenium

### Código de Scraping

- Primeiro, é preciso instalar o WebDriver para que o Selenium possa operar as automações em um navegador

- Para isso, será utilizada a biblioteca **WebDriver Managaer**, como explicado no artigo

- Queremos buscar a temperatura registrada  O link do local é uma página no climatempo que não possui as informações desejadas

- Para isso, primeiro vamos acessar a página do climatempo para o local usando o Selenium  

- É possível perceber que a informação desejada está na página "15 dias". Para acessar essa página, é preciso clinar no link correspondente, evidenciando a necessidade de um _scraping_ dinâmico com _Selenium_.

- Os dados desejados estão localizados em um gráficos e os dados que formam esse gráfico está dispostos no atributo _data-infos_.

- Para o dia do acesso, a previsão está no elemento de id _wrapper-chart-1_, enquanto as previsões para cada hora dos dois dias seguintes estão dos elementos _wrapper-chart-2_ e _wrapper-char-3_, respectivamente.

- O atributo _data-infos_ contém um vetor com todas as horas dia: de 00h até as 23h. Esse vetor tem a seguinte forma

```
[
   {
       "date":"2022-08-31 00:00:00",
       "temperature": { "temperature":11 },
   },
     ...
   {
       "date":"2022-08-31 23:00:00",
           "temperature":{ "temperature":11 },
   }
]     
```

In [4]:
def scrape_location(driver, local):
    driver.get(local['url'])
    dt_acesso = today()
    
    # achando o botão para a previsão de 15 dias e clicando
    predictions_15_days_btn = driver.find_element(By.ID, 'Botao_barra_navegacao_15_dias')
    driver.execute_script('arguments[0].click();', predictions_15_days_btn)
    
    for elementIndex in range(1, 4):
        with db as connection:
            elementId = f"wrapper-chart-{elementIndex}"
            element = driver.find_element(By.ID, elementId)

            data = json.loads(element.get_attribute("data-infos"))

            dt_previsao = data[0]['date'].split()[0]

            # Registrando um acesso para o dia e o local da previsão que está sendo feita
            try:
                id_acesso = db.registra_acesso(local['id'], dt_acesso, dt_previsao)
            except IntegrityError:
                print("Acesso já feito")
                continue

            for predictin_to_hour in data:
                horario_previsao = predictin_to_hour['date'].split()[1]
                temperature_value = predictin_to_hour['temperature']['temperature']
                db.registra_previsao(id_acesso, horario_previsao, temperature_value)

In [5]:
# Carregando os locais
with db as connection:
    locais = db.get_locais()
    
# Instanciando o driver com Web Driver Manager
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

for local in locais:
    scrape_location(driver, local)

driver.quit()

# Exportando o banco para CSV

In [ ]:
def export_db_to_csv(path_to_query_file):
    with open(path_to_query_file) as file:
        query = file.read()
    
    with db as connection:
        result = db.con.cursor().execute(query).fetchall()

In [ ]:
# Exporta informações das previsões de todos os horarios pra todos os locais
with open('queries/all_previsoes.sql') as file:
    query = file.read()

with db as connection:
    db_df = pd.read_sql_query(query, db.con)
    db_df.to_csv('database.csv', index=False)

for group, frame in db_df.groupby(by=['DT_PREVISAO', 'CIDADE', 'HORARIO']):
    tmps = frame['TEMPERATURA'].unique()
    qt_tmps = len(tmps)
    if qt_tmps > 1:
        print(f"{group} teve {tmps}")